In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Load data from CSV
file_path = 'file3.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Ensure 'commentText' column contains strings
data['commentText'] = data['commentText'].astype(str)

# Function to convert sentiment labels to numerical values
sentiment_dict = {
    'Positive': 0,
    'Negative': 1,
    'Not_relevant': 2,
    'Mixed_feelings': 3,
    'Nuetral': 4
}

data['Sentiment_Class'] = data['Sentiment_Class'].map(sentiment_dict)

# Tokenization and sequence padding
max_words = 1000
max_seq_length = 100
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(data['commentText'])

sequences = tokenizer.texts_to_sequences(data['commentText'])
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post', truncating='post')

# Prepare target variable
labels = np.asarray(data['Sentiment_Class'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.1, random_state=42)

# Build Bi-GRU model
model = Sequential()
model.add(Embedding(max_words, 16, input_length=max_seq_length))
model.add(Bidirectional(GRU(32)))
model.add(Dense(5, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=callbacks)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/10
134/134 [==============================] - 7s 30ms/step - loss: 1.4061 - accuracy: 0.3965 - val_loss: 1.3191 - val_accuracy: 0.4704
Epoch 2/10
134/134 [==============================] - 3s 25ms/step - loss: 1.1874 - accuracy: 0.5282 - val_loss: 1.1302 - val_accuracy: 0.5413
Epoch 3/10
134/134 [==============================] - 3s 26ms/step - loss: 0.9725 - accuracy: 0.6192 - val_loss: 0.9859 - val_accuracy: 0.6141
Epoch 4/10
134/134 [==============================] - 3s 26ms/step - loss: 0.8859 - accuracy: 0.6543 - val_loss: 0.9873 - val_accuracy: 0.6169
Epoch 5/10
134/134 [==============================] - 3s 26ms/step - loss: 0.8355 - accuracy: 0.6748 - val_loss: 0.9619 - val_accuracy: 0.6305
Epoch 6/10
134/134 [==============================] - 4s 26ms/step - loss: 0.8063 - accuracy: 0.6884 - val_loss: 0.9559 - val_accuracy: 0.6366
Epoch 7/10
134/134 [==============================] - 3s 26ms/step - loss: 0.7836 - accuracy: 0.7020 - val_loss: 0.9547 - val_accuracy: 0.6380